In [3]:
from datasets import load_dataset

In [33]:
def concat(x, validation=False):
    concated = ""
    for k, v in x.items():
        if k in ["spoiler", "provenance", "spoilerPositions"]:
            continue
        concated += k + ": " + str(v) + "\n"
    if not validation:
        concated += "spoiler" + ": " + x["spoiler"] + "\n"
    else:
        concated += "spoiler" + ": "
    return concated.replace("???", "")

In [9]:
train_data_path = "train_changed.json"
train_data = load_dataset("json", data_files=train_data_path, split="train")
train_data = train_data.map(lambda x: {"text": concat(x)})

Found cached dataset json (/Users/eren/.cache/huggingface/datasets/json/default-c63e742358a6e02d/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)
Loading cached processed dataset at /Users/eren/.cache/huggingface/datasets/json/default-c63e742358a6e02d/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-48ebbcd949ba8f47.arrow


In [24]:
train_data["text"][10]

'postText: #TeenMom2 star @PBandJenelley_1 reveals the sex of her second child through social media\npostPlatform: Twitter\ntargetParagraphs: "Teen Mom 2" star Jenelle Evans took to Twitter and Instagram Monday (Feb. 3) to announce that she and boyfriend Nathan Griffin are expecting a baby boy. Evans has faced criticism for the pregnancy, largely because she does not currently have custody of her first child, 4-year-old Jace. In the Season 5 premiere of "Teen Mom 2," Evans had an abortion, which led to even more speculation about whether or not she is ready to have a second child. The reality TV star has a long history battling drug addiction, and has been arrested several times. Evans told In Touch that she had the abortion at the beginning of Season 5 because she was too "skinny and unhealthy" to continue with the unexpected pregnancy while in recovery for heroin addiction. Evans is legally married to Courtland Rogers, though the couple is separated. The father of her first son, Jace

In [34]:
validation_data_path = "validation_changed.json"
validation_data = load_dataset("json", data_files=validation_data_path, split="train")
validation_data = validation_data.map(lambda x: {"text": concat(x, validation=True)})

Found cached dataset json (/Users/eren/.cache/huggingface/datasets/json/default-6efd8ac4534909a1/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [35]:
validation_data["text"][0]

'postText: Five Nights at Freddy’s Sequel Delayed for Weird Reason\npostPlatform: Facebook\ntargetParagraphs: Five Nights at Freddy’s creator Scott Cawthon takes to Steam to tease a possible delay for Five Nights at Freddy’s: Sister Location, the fifth game in the series. For the past couple of years, horror gaming fans have been able to look forward to one new entry in the Five Nights at Freddy’s series after another, with four core games, one RPG spinoff, and a novel released so far. The next game in the franchise, Five Nights at Freddy’s: Sister Location, was scheduled to release this coming Friday, October 7th, but if developer Scott Cawthon is to be believed, the project has been delayed by a few months. According to a post by Cawthon on the Five Nights at Freddy’s: Sister Location Steam page, the game is being delayed because it’s too dark. Cawthon said that some of the plot elements are so disturbing that they are making him feel sick, and so he is thinking about delaying the ga

In [15]:
import openai

In [18]:
openai.api_key = "" #Removed

In [54]:
from tqdm import tqdm
import time
import json

In [40]:
results = {}

In [65]:
time.sleep(60)

In [66]:
c = 3
id = 0
for i, text in tqdm(enumerate(validation_data["text"][id:])):
    idx = id + i
    spoiler = validation_data["spoiler"][idx]

    prompt = "Input 1:\n"
    prompt += "{\n" + train_data["text"][6] + "}\n\n"
    prompt += "Input 2:\n"
    prompt += "{\n" + train_data["text"][10] + "}\n\n"
    prompt += "Input 3:\n"
    prompt += "{\n" + text

    chat_completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", 
                                               messages=[{"role": "user", 
                                                          "content": prompt}])
    results[idx] = {}
    results[idx]["gpt"] = chat_completion
    results[idx]["real_spoiler"] = spoiler
    c -= 1
    if c <= 0:# Due to request rate limit per seconds
        c = 3
        time.sleep(60)
    else:
        time.sleep(2)

72it [29:21, 24.47s/it]


In [69]:
file_path = "gpt-3.5-turbo-eval.json"
with open(file_path, "w") as json_file:
    json.dump(results, json_file, indent=4)

In [70]:
with open(file_path, "r") as json_file:
    data = json.load(json_file)